<a href="https://colab.research.google.com/github/GHAZ25/ImersaoIA/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Seja muito bem vindo ao reino de Zorzium!**

Que tal desfrutar de uma aventura feita *inteiramente com IA*?

Aqui, iremos criar via programação um cenária totalmente interativo, inspirado em jogos de RPG, com direito a batalhas, uso de itens e muito mais.

Para maiores detalhes, acesse https://github.com/GHAZ25/ImersaoIA

Vamos começar?

In [ ]:
# Aqui estão todos os módulos necessários para jogarmos nosso jogo.
from google.colab import userdata
import google.generativeai as genai
import pandas as pd
import requests
import json
import re
import os
import PIL.Image

# Como principal ferramenta, iremos usar a IA Gemini da Google. Lembre de gerar sua API KEY a partir de sua conta Google para usar esse recurso.
# Para mais informações, acesse: https://aistudio.google.com/app/apikey
API_KEY = userdata.get("API_KEY")
genai.configure(api_key = API_KEY)

Primeiro, temos que iniciar nosso modelo generativo.

Caso tenha curiosidade para conhecer, use também o Gemini diretamente em seu navegador.

https://gemini.google.com/


In [ ]:
# Primeiramente, vamos inicializar nosso modelo generativo com alguns parâmetros.
# Para saber mais, acesse: https://ai.google.dev/docs
generation_config = {
  "temperature": 1,
  "top_p": 0.9,
  "top_k": 0,
  "max_output_tokens": 2048
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

# Aqui está um ponto importante. Iremos fornecer algumas informações sobre como a IA deve se comportar durante nosso jogo.
# Mais informações serão dadas adiante.
system_instruction = """Aja como o mestre de um jogo de RPG.
Primeiramente, serão passadas instruções sobre a história do jogo e seu funcionamento, por isso guarde essas informações.
Durante o jogo serão aceitas apenas ações humanas possíveis dentro do contexto informado, como por exemplo olhar para os lados, caminhar ou erguer um objeto com peso plausível.
Caso a pergunta não siga o contexto do jogo dê uma mensagem gentil que informe ao jogador que isso não é possível. Assim que o jogador atingir o objetivo informe que o jogo acabou e pergunte se quer começar outra aventura."""

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

Para tornar funcional o jogo, será passado um conjunto de instruções usados como base.

No contexto da IA, irei o usar o que é conhecido como 'few-shot' para explicar o que quero que seja usado, porém como pretendo deixar o mais interativo e criativo possível optei por não usar uma base própria de informações.

In [ ]:
# Todo o projeto está disponível no GitHub em: https://github.com/GHAZ25/ImersaoIA
# Aqui, iremos fazer a leitura de tudo o que precisamos fazer para dar mais instruções.
resposta = requests.get("https://raw.githubusercontent.com/GHAZ25/ImersaoIA/main/aventura.json")

if resposta.status_code == 200:
  dados = resposta.json()
else:
  print("Desculpe, não foi possível carregar as instruções")
  exit()

# Para usarmos de uma forma efetiva, será carregado um conjunto de instruções adicionais sobre nosso jogo.
history = dados["instrucoes"]

history.append({"role": "user", "parts": ["Aqui estão os dados iniciais do personagem do jogo.", f"{dados['personagem']}"]})
history.append({"role": "user", "parts": ["Aqui estão todos itens disponíveis no jogo.", f"{dados['itens']}"]})
history.append({"role": "user", "parts": ["Aqui estão alguns equipamentos disponíveis no jogo. Com base neles, crie mais 100 novos.", f"{dados['equipamentos']}"]})

# Mas que tal irmos um pouco além? Muito além de textos podemos passar VÁRIOS FORMATOS DE ARQUIVO para interagir.
# Nesse ponto, vamos passar algumas criaturas usadas como referência.
for criatura in dados["criaturas"]:
    arquivo_imagem = criatura["imagem"].split("/")[-1]
    resposta = requests.get(criatura["imagem"])

    if resposta.status_code == 200:
        with open(arquivo_imagem, "wb") as temp:
            temp.write(resposta.content)
            history.append({"role": "user", "parts": [PIL.Image.open(arquivo_imagem), f"{criatura['atributos']}"]})

history.append({"role": "user", "parts": ["Foram enviadas 5 criaturas como referência. Crie mais 15 com base no conteúdo anterior e cuide para os locais onde possam aparecer sejam apenas os existentes na história."]})

# Tudo pronto! Vamos jogar?
chat = model.start_chat(history=history)

for criatura in dados["criaturas"]:
    arquivo_imagem = criatura["imagem"].split("/")[-1]
    os.remove(arquivo_imagem)

## **Hora de jogar!**

Execute o código abaixo para ter uma experiência de um jogo de RPG novo a cada momento. Converse com as pessoas, olhe o ambiente em volta e conheça novos lugares!

Como dica, você pode fazer alguns pedidos que terão um conteúdo totalmente único.


*   Peça para ver seus itens
*   Peça para ver seus equipamentos
*   Pergunte sobre seu progresso

In [ ]:
pergunta = ""
resposta = ""

# É aqui que a mágica acontece! Interaja com um mestre de uma mesa de RPG no melhor estilo "jogo de texto para computador dos anos 80".
try:
  print("-" * 200)
  chat.send_message("Faça um resumo sobre a história do jogo. Depois pergunte sobre o personagem e questione se quer alterar suas características.")

  try:
    valida_retorno = re.search("^(```json)", chat.last.text)

    if valida_retorno == None:
      retorno = json.loads(chat.last.text)
    else:
      retorno = json.loads(chat.last.text[8:len(chat.last.text) - 3])

    resposta = retorno["mensagem"]
  except:
    resposta = chat.last.text

  print(resposta, "\n")

  while resposta != "FIM_DE_JOGO" and pergunta != "SAIR":
    pergunta = input("\n - Resposta:\n")
    print("-" * 200)

    if pergunta == "SAIR":
      continue

    chat.send_message(pergunta)

    try:
      valida_retorno = re.search("^(```json)", chat.last.text)

      if valida_retorno == None:
        retorno = json.loads(chat.last.text)
      else:
        retorno = json.loads(chat.last.text[8:len(chat.last.text) - 3])

      resposta = retorno["mensagem"]
    except:
      resposta = chat.last.text

    if resposta == "FIM_DE_JOGO":
      continue
    elif resposta == "MOSTRAR_INFORMACOES":
      df = pd.DataFrame(retorno["extra"])
      print("-" * 100)
      print(df)
      print("-" * 100)
      print("\nO que deseja fazer?")
    else:
      print(resposta, "\n")
except Exception as ex:
  print("Exception: ", ex)
  print(f"ERRO: {chat.last.text}")